In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

# --- 因为本 Notebook 与数据文件在同一目录下，直接加载即可 ---
clean_data_file = 'a_stock_daily_data_clean.parquet'

try:
    df = pd.read_parquet(clean_data_file)
    print(f"✅ 数据加载成功！文件: {clean_data_file}")
    print(f"   共加载 {len(df)} 条数据。")
except FileNotFoundError:
    print(f"❌ 错误：找不到已清洗的数据文件 '{clean_data_file}'！")
    print("   请确保你已经成功运行了 data_cleaner 中的 run_cleaning_pipeline.py 脚本。")

# 显示数据的前几行，对数据有个初步印象
df.head()

✅ 数据加载成功！文件: a_stock_daily_data_clean.parquet
   共加载 10216839 条数据。


,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,name,list_date
0,2010-01-04,24.52,24.58,23.68,23.71,24.37,-0.66,-2.71,241922.76,5.802495e+05,平安银行,1991-04-03
1,2010-01-05,23.75,23.90,22.75,23.30,23.71,-0.41,-1.73,556499.82,1.293477e+06,平安银行,1991-04-03
2,2010-01-06,23.25,23.25,22.72,22.90,23.30,-0.40,-1.72,412143.13,9.444537e+05,平安银行,1991-04-03
3,2010-01-07,22.90,23.05,22.40,22.65,22.90,-0.25,-1.09,355336.85,8.041663e+05,平安银行,1991-04-03
4,2010-01-08,22.50,22.75,22.35,22.60,22.65,-0.05,-0.22,288543.06,6.506674e+05,平安银行,1991-04-03


In [3]:
# --- 验证1: 检查是否还存在ST股票 ---
print("正在检查ST股票...")
# 依据《因子投资：方法与实践》3.1.4节的黑名单规则
st_stocks = df[df['name'].str.contains('ST')]

if st_stocks.empty:
    print("✅ ST股票已全部剔除，验证通过！")
else:
    print("❌ 验证失败：数据中仍存在ST股票！")
    print("残留的ST股票列表：")
    print(st_stocks['name'].unique())

正在检查ST股票...
✅ ST股票已全部剔除，验证通过！


In [4]:
# --- 验证2: 检查是否还存在上市不足一年的次新股 ---
print("正在检查次新股...")
# 依据《因子投资：方法与实践》3.1.4节的黑名单规则
# 重新计算上市天数用于验证
df['days_on_market'] = (df['trade_date'] - df['list_date']).dt.days

# 寻找上市天数小于等于365的记录
new_stocks = df[df['days_on_market'] <= 365]

if new_stocks.empty:
    print("✅ 次新股已全部剔除，验证通过！")
else:
    print("❌ 验证失败：数据中仍存在上市不足一年的股票！")
    print("残留的次新股记录示例：")
    print(new_stocks[['ts_code', 'trade_date', 'list_date', 'days_on_market']].head())

# 删除辅助列
df = df.drop(columns=['days_on_market'])

正在检查次新股...
✅ 次新股已全部剔除，验证通过！


In [5]:
# --- 验证3: 检查离群值处理（收益率压缩） ---
print("正在检查 `pct_chg` 列的范围...")
# 依据《因子投资：方法与实践》3.1.2节的处理方法
min_pct_chg = df['pct_chg'].min()
max_pct_chg = df['pct_chg'].max()

print(f"收益率范围：[{min_pct_chg:.2f}%, {max_pct_chg:.2f}%]")

if min_pct_chg >= -10 and max_pct_chg <= 10:
    print("✅ 离群值处理验证通过，所有收益率均在 [-10, 10] 区间内！")
else:
    print("❌ 验证失败：存在超出范围的收益率！")

# 也可以用 describe() 做一个整体的描述性统计概览
df[['pct_chg']].describe()

正在检查 `pct_chg` 列的范围...
收益率范围：[-10.00%, 10.00%]
✅ 离群值处理验证通过，所有收益率均在 [-10, 10] 区间内！


,pct_chg
count,1.021684e+07
mean,5.219337e-02
std,2.897599e+00
min,-1.000000e+01
25%,-1.390900e+00
50%,0.000000e+00
75%,1.360000e+00
max,1.000000e+01


In [6]:
# --- 验证4: 检查缺失值 ---
print("正在检查是否存在缺失值...")
# 依据《因子投资：方法与实践》3.1.2节的处理方法
missing_values = df.isnull().sum()

# 筛选出仍然存在缺失值的列
missing_cols = missing_values[missing_values > 0]

if missing_cols.empty:
    print("✅ 缺失值处理验证通过，数据集中没有NaN值！")
else:
    print("❌ 验证失败：以下列仍然存在缺失值！")
    print(missing_cols)
    
# 使用.info()可以更全面地查看每一列的数据类型和非空值数量
print("\n--- DataFrame.info() ---")
df.info()

正在检查是否存在缺失值...
✅ 缺失值处理验证通过，数据集中没有NaN值！

--- DataFrame.info() ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10216839 entries, 0 to 10216838
Data columns (total 12 columns):
 #   Column      Dtype         
---  ------      -----         
 0   trade_date  datetime64[ns]
 1   open        float64       
 2   high        float64       
 3   low         float64       
 4   close       float64       
 5   pre_close   float64       
 6   change      float64       
 7   pct_chg     float64       
 8   vol         float64       
 9   amount      float64       
 10  name        object        
 11  list_date   datetime64[ns]
dtypes: datetime64[ns](2), float64(9), object(1)
memory usage: 935.4+ MB


In [7]:
# --- 验证4: 检查指数数据---
clean_data_file = 'a_index_daily_data_clean.parquet'

try:
    df = pd.read_parquet(clean_data_file)
    print(f"✅ 数据加载成功！文件: {clean_data_file}")
    print(f"   共加载 {len(df)} 条数据。")
except FileNotFoundError:
    print(f"❌ 错误：找不到已清洗的数据文件 '{clean_data_file}'！")
    print("   请确保你已经成功运行了 data_cleaner 中的 run_cleaning_pipeline.py 脚本。")

# 显示数据的前几行，对数据有个初步印象
df.head()

✅ 数据加载成功！文件: a_index_daily_data_clean.parquet
   共加载 22956 条数据。


,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,2010-01-04,3243.760,3289.750,3295.279,3243.319,3277.139,-33.379,-1.0185,109447927.0,1.337735e+08
1,2010-01-05,3282.179,3254.468,3290.512,3221.462,3243.760,38.419,1.1844,126115066.0,1.618582e+08
2,2010-01-06,3254.215,3277.517,3295.868,3253.044,3282.179,-27.964,-0.8520,123651384.0,1.580401e+08
3,2010-01-07,3192.776,3253.991,3268.819,3176.707,3254.215,-61.439,-1.8880,128652827.0,1.572296e+08
4,2010-01-08,3195.997,3177.259,3198.920,3149.017,3192.776,3.221,0.1009,98377147.0,1.217399e+08


In [12]:
# --- 验证4: 检查资产负债表数据---
clean_data_file = 'a_stock_balancesheet_data_clean.parquet'

try:
    df = pd.read_parquet(clean_data_file)
    print(f"✅ 数据加载成功！文件: {clean_data_file}")
    print(f"   共加载 {len(df)} 条数据。")
except FileNotFoundError:
    print(f"❌ 错误：找不到已清洗的数据文件 '{clean_data_file}'！")
    print("   请确保你已经成功运行了 data_cleaner 中的 run_cleaning_pipeline.py 脚本。")

# 显示数据的前几行，对数据有个初步印象
print(df.columns)

✅ 数据加载成功！文件: a_stock_balancesheet_data_clean.parquet
   共加载 356393 条数据。
Index(['ts_code', 'ann_date', 'f_ann_date', 'end_date', 'report_type',
       'comp_type', 'end_type', 'total_share', 'cap_rese', 'undistr_porfit',
       ...
       'accounts_receiv_bill', 'accounts_pay', 'oth_rcv_total',
       'fix_assets_total', 'cip_total', 'oth_pay_total', 'long_pay_total',
       'debt_invest', 'oth_debt_invest', 'update_flag'],
      dtype='object', length=152)


In [9]:
# --- 验证5: 检查现金流量表数据---
clean_data_file = 'a_stock_cashflow_data_clean.parquet'

try:
    df = pd.read_parquet(clean_data_file)
    print(f"✅ 数据加载成功！文件: {clean_data_file}")
    print(f"   共加载 {len(df)} 条数据。")
except FileNotFoundError:
    print(f"❌ 错误：找不到已清洗的数据文件 '{clean_data_file}'！")
    print("   请确保你已经成功运行了 data_cleaner 中的 run_cleaning_pipeline.py 脚本。")

# 显示数据的前几行，对数据有个初步印象
df.head()

✅ 数据加载成功！文件: a_stock_cashflow_data_clean.parquet
   共加载 338328 条数据。


,ts_code,ann_date,f_ann_date,end_date,comp_type,report_type,end_type,net_profit,finan_exp,c_fr_sale_sg,...,net_dism_capital_add,net_cash_rece_sec,credit_impa_loss,use_right_asset_dep,oth_loss_asset,end_bal_cash,beg_bal_cash,end_bal_cash_equ,beg_bal_cash_equ,update_flag
0,000001.SZ,20250823,20250823,20250630,2,1,2,4.880739e+07,0.0,0.0,...,2.835000e+09,0.0,1.938500e+10,9.380000e+08,65000000.0,1.591034e+08,1.400400e+08,3.009440e+11,2.535250e+11,0
1,000001.SZ,20250419,20250419,20250331,2,1,1,0.000000e+00,0.0,0.0,...,1.190700e+10,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0
2,000001.SZ,20250315,20250315,20241231,2,1,4,4.880739e+07,0.0,0.0,...,6.251600e+10,0.0,4.893900e+10,2.247000e+09,0.0,1.591034e+08,1.400400e+08,2.535250e+11,2.945320e+11,0
3,000001.SZ,20250315,20250315,20241231,2,1,4,4.880739e+07,0.0,0.0,...,6.251600e+10,0.0,4.893900e+10,2.247000e+09,489000000.0,1.591034e+08,1.400400e+08,2.535250e+11,2.945320e+11,1
4,000001.SZ,20241019,20241019,20240930,2,1,3,0.000000e+00,0.0,0.0,...,4.802600e+10,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0


In [10]:
# --- 验证6: 检查收益表数据---
clean_data_file = 'a_stock_income_data_clean.parquet'

try:
    df = pd.read_parquet(clean_data_file)
    print(f"✅ 数据加载成功！文件: {clean_data_file}")
    print(f"   共加载 {len(df)} 条数据。")
except FileNotFoundError:
    print(f"❌ 错误：找不到已清洗的数据文件 '{clean_data_file}'！")
    print("   请确保你已经成功运行了 data_cleaner 中的 run_cleaning_pipeline.py 脚本。")

# 显示数据的前几行，对数据有个初步印象
df

✅ 数据加载成功！文件: a_stock_income_data_clean.parquet
   共加载 362154 条数据。


,ts_code,ann_date,f_ann_date,end_date,report_type,comp_type,end_type,basic_eps,diluted_eps,total_revenue,...,withdra_biz_devfund,withdra_rese_fund,withdra_oth_ersu,workers_welfare,distr_profit_shrhder,prfshare_payable_dvd,comshare_payable_dvd,capit_comstock_div,continued_net_profit,update_flag
0,000001.SZ,20250823,20250823,20250630,1,2,2,0.762692,0.708214,3.399053e+09,...,None,None,0.0,0.0,0.0,0.0,0.0,0.0,5.625280e+07,1
1,000001.SZ,20250419,20250419,20250331,1,2,1,0.620000,0.620000,3.399053e+09,...,None,None,0.0,0.0,0.0,0.0,0.0,0.0,5.625280e+07,1
2,000001.SZ,20250315,20250315,20241231,1,2,4,0.762692,0.708214,3.399053e+09,...,None,None,0.0,0.0,0.0,0.0,0.0,0.0,5.625280e+07,1
3,000001.SZ,20241019,20241019,20240930,1,2,3,0.762692,0.708214,3.399053e+09,...,None,None,0.0,0.0,0.0,0.0,0.0,0.0,5.625280e+07,1
4,000001.SZ,20240816,20240816,20240630,1,2,2,0.762692,0.708214,3.399053e+09,...,None,None,0.0,0.0,0.0,0.0,0.0,0.0,5.625280e+07,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362149,689009.SH,20230331,20230331,20221231,1,1,4,0.762692,0.708214,3.399053e+09,...,None,None,0.0,0.0,0.0,0.0,0.0,0.0,5.625280e+07,1
362150,689009.SH,20220426,20220426,20211231,1,1,4,0.762692,0.708214,3.399053e+09,...,None,None,0.0,0.0,0.0,0.0,0.0,0.0,5.625280e+07,0
362151,689009.SH,20210416,20210416,20201231,1,1,4,0.762692,0.708214,3.399053e+09,...,None,None,0.0,0.0,0.0,0.0,0.0,0.0,5.625280e+07,0
362152,689009.SH,20200930,20200930,20200630,1,1,2,-0.482692,0.000000,2.001800e+09,...,None,None,0.0,0.0,0.0,0.0,0.0,0.0,-5.625280e+07,0


In [13]:
# --- 验证5: 检查现金流量表数据---
raw_data_file = '../raw_data/a_stock_cashflow_data_clean.parquet'

try:
    df = pd.read_parquet(raw_data_file)
    print(f"✅ 数据加载成功！文件: {raw_data_file}")
    print(f"   共加载 {len(df)} 条数据。")
except FileNotFoundError:
    print(f"❌ 错误：找不到已清洗的数据文件 '{raw_data_file}'！")

# 显示数据的前几行，对数据有个初步印象
df

❌ 错误：找不到已清洗的数据文件 '../raw_data/a_stock_cashflow_data_clean.parquet'！


,ts_code,ann_date,f_ann_date,end_date,report_type,comp_type,end_type,total_share,cap_rese,undistr_porfit,...,accounts_receiv_bill,accounts_pay,oth_rcv_total,fix_assets_total,cip_total,oth_pay_total,long_pay_total,debt_invest,oth_debt_invest,update_flag
0,000001.SZ,20250823,20250823,20250630,1,2,None,1.655173e+09,2.658589e+09,1.856439e+09,...,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0
1,000001.SZ,20250823,20250823,20250630,1,2,2,1.655173e+09,2.658589e+09,1.856439e+09,...,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,7.513720e+11,1.806120e+11,1
2,000001.SZ,20250419,20250419,20250331,1,2,1,1.655173e+09,2.658589e+09,1.856439e+09,...,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,7.776340e+11,1.734530e+11,1
3,000001.SZ,20250315,20250315,20241231,1,2,4,1.655173e+09,2.658589e+09,1.856439e+09,...,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,7.850750e+11,1.766550e+11,1
4,000001.SZ,20241019,20241019,20240930,1,2,3,1.655173e+09,2.658589e+09,1.856439e+09,...,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,7.774030e+11,1.797940e+11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356388,689009.SH,20230331,20230331,20221231,1,1,4,7.129831e+07,2.658589e+09,-1.111390e+09,...,1.184620e+09,1.210235e+09,68069691.61,9.367502e+08,1.023987e+08,2.433909e+08,24800000.0,0.000000e+00,0.000000e+00,0
356389,689009.SH,20220426,20220426,20211231,1,1,4,7.078872e+07,2.658589e+09,-1.111390e+09,...,7.709374e+08,1.210235e+09,33514101.77,5.894623e+08,7.792133e+07,1.753098e+08,24800000.0,0.000000e+00,0.000000e+00,0
356390,689009.SH,20210416,20210416,20201231,1,1,4,7.040917e+07,2.658589e+09,-1.111390e+09,...,7.566453e+08,1.210235e+09,39336649.34,4.893379e+08,1.013861e+07,1.555185e+08,24800000.0,0.000000e+00,0.000000e+00,0
356391,689009.SH,20200930,20200930,20200630,1,1,2,6.336825e+07,2.658589e+09,-1.111390e+09,...,7.553099e+08,1.210235e+09,45021353.41,2.513755e+08,1.552839e+08,1.125706e+08,24800000.0,0.000000e+00,0.000000e+00,0


In [ ]:
# 通用数据加载函数
def load_data_file(data_type, cleaned=False):
    """
    通用数据加载函数
    
    Args:
        data_type (str): 数据类型，可选值:
            - 'daily': 股票日度数据
            - 'cashflow': 现金流量表数据  
            - 'balancesheet': 资产负债表数据
            - 'income': 利润表数据
            - 'index': 指数数据
        cleaned (bool): True加载清洗后数据，False加载原始数据
    
    Returns:
        pd.DataFrame: 加载的数据
    """
    
    # 文件映射
    file_mapping = {
        'daily': 'a_stock_daily_data',
        'cashflow': 'a_stock_cashflow_data', 
        'balancesheet': 'a_stock_balancesheet_data',
        'income': 'a_stock_income_data',
        'index': 'a_index_daily_data'
    }
    
    if data_type not in file_mapping:
        raise ValueError(f"不支持的数据类型: {data_type}. 支持的类型: {list(file_mapping.keys())}")
    
    base_filename = file_mapping[data_type]
    
    if cleaned:
        # 加载清洗后的数据 (在当前目录 clean_data 中)
        filename = f"{base_filename}_clean.parquet"
        filepath = filename
    else:
        # 加载原始数据 (在上级目录的 raw_data 中)
        filename = f"{base_filename}.parquet"
        filepath = f"../raw_data/{filename}"
    
    try:
        df = pd.read_parquet(filepath)
        status = "清洗后" if cleaned else "原始"
        print(f"✅ {status}数据加载成功！")
        print(f"   数据类型: {data_type}")
        print(f"   文件路径: {filepath}")
        print(f"   数据量: {len(df)} 条记录，{len(df.columns)} 列")
        return df
    except FileNotFoundError:
        print(f" 错误：找不到文件 '{filepath}'")
        return None
    except Exception as e:
        print(f" 加载出错: {e}")
        return None

# 示例用法
print("=== 使用通用函数加载数据示例 ===")

# 加载原始现金流量表数据
df_cashflow_raw = load_data_file('cashflow', cleaned=False)

print("\n" + "-"*50)

# 尝试加载清洗后的现金流量表数据
df_cashflow_clean = load_data_file('cashflow', cleaned=True)

=== 使用通用函数加载数据示例 ===
✅ 原始数据加载成功！
   数据类型: cashflow
   文件路径: ../raw_data/a_stock_cashflow_data.parquet
   数据量: 338328 条记录，97 列

--------------------------------------------------
✅ 清洗后数据加载成功！
   数据类型: cashflow
   文件路径: a_stock_cashflow_data_clean.parquet
   数据量: 338328 条记录，97 列


In [18]:
# --- 演示加载各种类型的数据 ---
print("=== 演示通用函数加载所有数据类型 ===\n")

# 1. 加载日线行情数据（最常用的数据）
print("1️⃣ 加载股票日线行情数据:")
df_daily = load_data_file('daily', cleaned=True)
if df_daily is not None:
    print(f"   所有列: {list(df_daily.columns)}")  # 显示所有列名
    print(f"   时间范围: {df_daily['trade_date'].min()} 至 {df_daily['trade_date'].max()}")
    # 检查是否有股票代码列
    if 'ts_code' in df_daily.columns:
        print(f"   股票数量: {df_daily['ts_code'].nunique()} 只")
    elif 'code' in df_daily.columns:
        print(f"   股票数量: {df_daily['code'].nunique()} 只")
    else:
        print(f"   数据行数: {len(df_daily)} 条")

print("\n" + "-"*50)

# 2. 加载指数数据
print("2️⃣ 加载指数日线数据:")
df_index = load_data_file('index', cleaned=True)
if df_index is not None:
    print(f"   所有列: {list(df_index.columns)}")
    if 'trade_date' in df_index.columns:
        print(f"   时间范围: {df_index['trade_date'].min()} 至 {df_index['trade_date'].max()}")
    # 检查指数代码列
    for col in ['ts_code', 'code', 'index_code']:
        if col in df_index.columns:
            print(f"   指数数量: {df_index[col].nunique()} 个")
            break

print("\n" + "-"*50)

# 3. 加载财务数据示例
print("3️⃣ 加载资产负债表数据:")
df_balance = load_data_file('balancesheet', cleaned=True)
if df_balance is not None:
    print(f"   财务科目数量: {len(df_balance.columns)} 个")
    print(f"   前10列: {list(df_balance.columns[:10])}")

print("\n" + "-"*50)

# 4. 加载利润表数据
print("4️⃣ 加载利润表数据:")
df_income = load_data_file('income', cleaned=True)
if df_income is not None:
    print(f"   财务科目数量: {len(df_income.columns)} 个")
    print(f"   前10列: {list(df_income.columns[:10])}")

print("\n" + "="*50)
print("📊 总结: 通用函数支持加载以下所有数据类型:")
print("   • 'daily': 股票日线行情数据 (价格、成交量等)")
print("   • 'index': 指数日线数据") 
print("   • 'cashflow': 现金流量表数据")
print("   • 'balancesheet': 资产负债表数据") 
print("   • 'income': 利润表数据")
print("\n💡 用法: load_data_file('数据类型', cleaned=True/False)")

=== 演示通用函数加载所有数据类型 ===

1️⃣ 加载股票日线行情数据:
✅ 清洗后数据加载成功！
   数据类型: daily
   文件路径: a_stock_daily_data_clean.parquet
   数据量: 10216839 条记录，12 列
   所有列: ['trade_date', 'open', 'high', 'low', 'close', 'pre_close', 'change', 'pct_chg', 'vol', 'amount', 'name', 'list_date']
   时间范围: 2010-01-04 00:00:00 至 2025-09-30 00:00:00
   数据行数: 10216839 条

--------------------------------------------------
2️⃣ 加载指数日线数据:
✅ 清洗后数据加载成功！
   数据类型: index
   文件路径: a_index_daily_data_clean.parquet
   数据量: 22956 条记录，10 列
   所有列: ['trade_date', 'close', 'open', 'high', 'low', 'pre_close', 'change', 'pct_chg', 'vol', 'amount']
   时间范围: 2010-01-04 00:00:00 至 2025-09-30 00:00:00

--------------------------------------------------
3️⃣ 加载资产负债表数据:
✅ 清洗后数据加载成功！
   数据类型: balancesheet
   文件路径: a_stock_balancesheet_data_clean.parquet
   数据量: 356393 条记录，152 列
   财务科目数量: 152 个
   前10列: ['ts_code', 'ann_date', 'f_ann_date', 'end_date', 'report_type', 'comp_type', 'end_type', 'total_share', 'cap_rese', 'undistr_porfit']

------------